In [42]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor, VotingClassifier,VotingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,mean_squared_error
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sklearn.metrics as metrics
import seaborn as sns

In [43]:
# Surpressing warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [44]:
df = pd.read_csv('train.csv')
df.head()

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온
0,1960-01-01,2.2,-5.2,7.4,NaN,68.3,1.7,6.7,NaN,NaN,-1.6
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,NaN,NaN,-1.9
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,NaN,NaN,4.0
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,NaN,NaN,7.5
4,1960-01-05,1.3,-8.2,9.5,NaN,44.0,5.1,8.2,NaN,NaN,-4.6


In [45]:

# 데이터프레임을 날짜 열을 기준으로 정렬합니다 (필요에 따라 생략 가능)
df = df.sort_values(by='일시')

# 2000년 이전의 데이터를 삭제합니다.
df = df[df['일시'] >= '2000-01-01']
# NaN 값을 0으로 채우기
bf = df.fillna(0)

# 결과 확인
print(bf)


               일시  최고기온  최저기온   일교차   강수량  평균습도  평균풍속  일조합    일사합   일조율  평균기온
14610  2000-01-01   9.9   1.8   8.1   0.0  63.5   2.3  2.3   4.46  24.0   5.5
14611  2000-01-02   6.9  -0.9   7.8   6.0  74.3   3.6  2.2   2.76  22.9   4.2
14612  2000-01-03   0.1  -4.6   4.7   0.0  54.1   2.7  5.4   5.11  56.3  -2.2
14613  2000-01-04   4.3  -4.3   8.6   0.0  48.6   1.7  1.6   3.59  16.7   0.3
14614  2000-01-05   4.6   0.1   4.5  18.4  83.6   3.2  0.0   0.69   0.0   2.8
...           ...   ...   ...   ...   ...   ...   ...  ...    ...   ...   ...
23006  2022-12-27   3.3  -7.3  10.6   0.0  69.8   1.8  8.8  10.25  91.7  -2.6
23007  2022-12-28   0.1  -6.0   6.1   0.1  58.1   2.5  8.7  10.86  90.6  -3.3
23008  2022-12-29   2.1  -7.8   9.9   0.0  56.3   1.7  9.0  10.88  93.8  -2.9
23009  2022-12-30   2.3  -4.4   6.7   0.0  65.6   1.9  7.9  10.84  82.3  -1.8
23010  2022-12-31   2.1  -5.1   7.2   0.0  65.5   1.4  1.1   4.16  11.5  -1.2

[8401 rows x 11 columns]


In [46]:
bf['일시'] = pd.to_datetime(bf['일시'])

bf['년'] = bf['일시'].dt.year
bf['월'] = bf['일시'].dt.month
bf['일'] = bf['일시'].dt.day

X = bf[['년', '월', '일']]
y = bf['평균기온']

In [47]:
print(y.shape)
X.iloc[6000]

(8401,)


년    2016
월       6
일       5
Name: 20610, dtype: int64

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)

StandardScaler()

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

x_train, x_test = X[:7670], X[7670:]
y_train, y_test = y[:7670], y[7670:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)


## Linear Regression

In [50]:
LinearReg = LinearRegression()
LinearReg.fit(X_train_scaled, y_train)

LinearRegression()

## Random Forest Regression Model

In [51]:
RFR = RandomForestRegressor()
RFR.fit(X_train_scaled, y_train)

RandomForestRegressor()

## SVM Regression Model

In [52]:
SVRR = SVR()
SVRR.fit(X_train_scaled, y_train)

SVR()

## Ensembled of the above Regression Models

In [53]:
ereg = VotingRegressor(estimators=[('LinearReg',LinearReg),('RandomForest', RFR), ('SVR',SVRR)],weights=[1,2,3])
ereg.fit(X_train_scaled, y_train)

VotingRegressor(estimators=[('LinearReg', LinearRegression()),
                            ('RandomForest', RandomForestRegressor()),
                            ('SVR', SVR())],
                weights=[1, 2, 3])

##  Regression report on rainfall amount prediction

In [54]:
LinearRegression_MAE = metrics.mean_absolute_error(LinearReg.predict(X_test_scaled), y_test)
LinearRegression_MSE = metrics.mean_squared_error(LinearReg.predict(X_test_scaled), y_test, squared=False)

RandomForest_MAE = metrics.mean_absolute_error(RFR.predict(X_test_scaled), y_test)
RandomForest_MSE = metrics.mean_squared_error(RFR.predict(X_test_scaled), y_test, squared=False)

SVM_MAE = metrics.mean_absolute_error(SVRR.predict(X_test_scaled), y_test)
SVM_MSE = metrics.mean_squared_error(SVRR.predict(X_test_scaled), y_test, squared=False)

ereg_MAE = metrics.mean_absolute_error(ereg.predict(X_test_scaled), y_test)
ereg_MSE = metrics.mean_squared_error(ereg.predict(X_test_scaled), y_test, squared=False)

Report = {'Regression Algorithm': ['Linear Regression','Random Forest', 'SVM','Ensemble Regression'],
          'MAE': [LinearRegression_MAE,RandomForest_MAE, SVM_MAE,ereg_MAE],
          'RMSE': [LinearRegression_MSE,RandomForest_MSE, SVM_MSE,ereg_MSE],}
Report = pd.DataFrame(Report)
Report

,Regression Algorithm,MAE,RMSE
0,Linear Regression,8.856779,10.500623
1,Random Forest,3.386598,4.380551
2,SVM,3.366302,4.166315
3,Ensemble Regression,3.764863,4.682148


In [55]:
y_pred = RFR.predict(X_test_scaled)

In [56]:
y_pred

array([-6.431, -1.259,  0.309,  0.052,  0.891,  1.74 ,  2.659,  4.793,
        2.688, -0.131, -0.55 , -0.391, -0.786, -2.329, -2.431, -2.458,
       -1.756, -0.831,  0.183,  1.552,  0.485,  0.163,  3.285,  3.939,
        3.282,  3.991,  5.118,  6.032,  6.477,  5.836,  5.342,  4.471,
        1.401,  1.051,  0.285, -1.85 , -6.147, -5.401, -2.46 , -0.777,
       -0.47 ,  3.101,  6.   ,  6.778,  6.694,  7.343,  7.009, -1.186,
       -4.014, -3.209,  1.294,  4.084,  5.613,  3.962,  3.578,  6.16 ,
        6.31 ,  6.203,  6.119,  5.05 ,  4.862,  4.257,  4.773,  3.496,
        2.843,  3.667,  6.14 ,  8.159,  9.662,  6.448,  4.934,  6.09 ,
        4.586,  4.367,  4.354,  4.382,  7.002,  9.314,  9.159,  9.662,
       11.451, 10.622, 10.305, 11.176, 11.901, 13.663, 11.695,  9.056,
        9.153,  9.707, 10.386,  9.621,  9.328,  9.647,  9.04 ,  8.358,
        9.06 , 10.298,  9.862,  9.268,  9.559,  9.96 , 10.402, 12.089,
       12.504, 13.359, 15.14 , 13.131, 13.182, 11.575,  9.303,  8.067,
      

In [57]:
fd= pd.read_csv('sample_submission.csv')
fd['평균기온'] = y_pred[-358:]
fd

,일시,평균기온
0,2023-01-01,2.688
1,2023-01-02,-0.131
2,2023-01-03,-0.550
3,2023-01-04,-0.391
4,2023-01-05,-0.786
...,...,...
353,2023-12-20,4.036
354,2023-12-21,4.867
355,2023-12-22,0.826
356,2023-12-23,-5.614


In [58]:
fd.to_csv("nagga_submission.csv", index=False)

In [24]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Bar(x=Report['Regression Algorithm'], y=Report['MAE'], name='MAE'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=Report['Regression Algorithm'], y=Report['RMSE'], name='RMSE'), row=1,col=1
)
fig.update_layout(title_text="Error of Regression Algorithms")
fig.update_yaxes(title_text="Error", row=1, col=1)
fig.update_xaxes(title_text="Regression Algorithm", row=1, col=1)
fig.show()

NameError: name 'make_subplots' is not defined

# Average Temperature of a day Prediction

In [31]:
df_proc = df_proc.astype(float)
df_proc['AvgTemp']=df_proc[['Temp9am', 'Temp3pm','MinTemp','MaxTemp']].mean(axis=1)
features = df_proc.drop(columns=['Temp9am', 'Temp3pm','MinTemp','MaxTemp','AvgTemp'], axis=1)
Y = df_proc['AvgTemp']
x_train, x_test, y_train, y_test = train_test_split(features, Y, test_size=0.35, random_state=101)

# Linear Regression

In [32]:
LinearReg = LinearRegression()
LinearReg.fit(x_train, y_train)

LinearRegression()

## Random Forest Regression Model

In [33]:
RFR = RandomForestRegressor()
RFR.fit(x_train, y_train)

RandomForestRegressor()

## SVM Regression Model

In [34]:
SVR=SVR()
SVR.fit(x_train, y_train)

SVR()

## Ensembled of the above Regression Models

In [35]:
ereg = VotingRegressor(estimators=[('LinearReg',LinearReg),('RandomForest', RFR), ('SVR',SVR)],weights=[1,2,3])
ereg.fit(x_train, y_train)

VotingRegressor(estimators=[('LinearReg', LinearRegression()),
                            ('RandomForest', RandomForestRegressor()),
                            ('SVR', SVR())],
                weights=[1, 2, 3])

##  Regression report on average temperature prediction

In [36]:
LinearRegression_MAE = metrics.mean_absolute_error(LinearReg.predict(x_test), y_test)
LinearRegression_MSE = metrics.mean_squared_error(LinearReg.predict(x_test), y_test, squared=False)

RandomForest_MAE = metrics.mean_absolute_error(RFR.predict(x_test), y_test)
RandomForest_MSE = metrics.mean_squared_error(RFR.predict(x_test), y_test, squared=False)

SVM_MAE = metrics.mean_absolute_error(SVR.predict(x_test), y_test)
SVM_MSE = metrics.mean_squared_error(SVR.predict(x_test), y_test, squared=False)

ereg_MAE = metrics.mean_absolute_error(ereg.predict(x_test), y_test)
ereg_MSE = metrics.mean_squared_error(ereg.predict(x_test), y_test, squared=False)

Report = {'Regression Algorithm': ['Linear Regression','Random Forest', 'SVM','Ensemble Regression'],
          'MAE': [LinearRegression_MAE,RandomForest_MAE, SVM_MAE,ereg_MAE],
          'RMSE': [LinearRegression_MSE,RandomForest_MSE, SVM_MSE,ereg_MSE],}
Report = pd.DataFrame(Report)
Report


,Regression Algorithm,MAE,RMSE
0,Linear Regression,0.470594,0.603209
1,Random Forest,0.449459,0.569627
2,SVM,0.434701,0.560317
3,Ensemble Regression,0.425699,0.545638


In [37]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Bar(x=Report['Regression Algorithm'], y=Report['MAE'], name='MAE'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=Report['Regression Algorithm'], y=Report['RMSE'], name='RMSE'), row=1,col=1
)
fig.update_layout(title_text="Error of Regression Algorithms")
fig.update_yaxes(title_text="Error", row=1, col=1)
fig.update_xaxes(title_text="Regression Algorithm", row=1, col=1)
fig.show()